In [321]:
import pandas as pd

from util import Pipeline

p = Pipeline('configs')

In [ ]:
with pd.HDFStore('data/pipeline.h5') as store:
    print(store.keys())

In [ ]:
# load control to target lookup
control_target_lookup = p.get_table('control_target_lookup')

# sum decennial data to target areas
dec = (
    p.get_table('decennial_by_control_area')
    .merge(control_target_lookup, on='control_id', how='left')
    .groupby(['target_id','RGID','county_id']).sum().reset_index()
    .drop(columns=['control_id','name'])
)

In [ ]:
# merge decennial data with adjusted targets
df = (
    p.get_table('adjusted_units_change_targets')
    .merge(dec, on='target_id', how='left')
)

In [ ]:
targets_rgid = df.drop(columns=['target_id','start','county_id']).groupby('RGID').sum().reset_index()

In [ ]:
king_hhsz = p.settings['king_hhsz']
king_vac_rates = p.settings['king_vac']


In [313]:
targets_rgid['hhsz'] = targets_rgid['RGID'].map(king_hhsz)

In [317]:
target_horizon_year = p.settings['target_horizon_year']
units_horizon_col = f'units_{target_horizon_year}'

targets_rgid[units_horizon_col] = targets_rgid['dec_units'] + targets_rgid['units_chg_adj']

households_horizon_col = f'hh_{target_horizon_year}'
targets_rgid['vacancy_rate'] = targets_rgid['RGID'].map(king_vac_rates)
targets_rgid[households_horizon_col] = (targets_rgid[units_horizon_col] * (1 - targets_rgid['vacancy_rate']/100)).round(0).astype(int)

In [319]:
hhpop_horizon_col = f'hhpop_{target_horizon_year}'
targets_rgid[hhpop_horizon_col] = (targets_rgid[households_horizon_col] * targets_rgid['hhsz']).round(0).astype(int)

In [324]:
hhpop_horizon_forced_total = p.settings['king_hhpop_2044']

In [327]:
hhpop_horizon_sum = targets_rgid[hhpop_horizon_col].sum()

In [330]:
hhpop_factor = hhpop_horizon_forced_total / hhpop_horizon_sum 

In [331]:
factored_hhpop_horizon_col = f'factored_hhpop_{target_horizon_year}'
targets_rgid[factored_hhpop_horizon_col] = (targets_rgid[hhpop_horizon_col] * hhpop_factor).round(0).astype(int)

In [335]:
# calculate units for target horizon year
df[units_horizon_col] = df['units_chg_adj'] + df['dec_units']

In [336]:
# calculate households for target horizon year

# get king vacancy rates from settings.yaml
df['vacancy_rate'] = df['RGID'].map(king_vac_rates)

# calculate households by applying vacancy rates to units

df[households_horizon_col] = (df[units_horizon_col] * (1 - df['vacancy_rate']/100)).round(0).astype(int)

In [337]:
# get king hhsz from settings.yaml
king_metro_adj_hhsz = p.settings['king_metro_adj_hhsz']
king_hhsz_adj = king_hhsz.copy()
king_hhsz_adj[1] = king_metro_adj_hhsz
df['king_hhsz'] = df['RGID'].map(king_hhsz_adj)

In [338]:
df['dec_hhpop_by_rgid'] = df.groupby('RGID')['dec_hhpop'].transform('sum')
df['dec_hh_by_rgid'] = df.groupby('RGID')['dec_hh'].transform('sum')
df['dec_hhsz_by_rgid'] = df['dec_hhpop_by_rgid'] / df['dec_hh_by_rgid']

In [339]:
# calculate decennial hhsz
df['dec_hhsz'] = df['dec_hhpop'] / df['dec_hh']

# calculate adjusted hhsz
df['hhsz'] = df['king_hhsz'] / df['dec_hhsz_by_rgid'] * df['dec_hhsz']

# if hhsz is greater than 5, use original value
df.loc[df.hhsz>5, 'hhsz'] = df.loc[df.hhsz>5, 'king_hhsz']

In [340]:
# calculate hhpop for target horizon year
hhpop_horizon_col = f'hhpop_{target_horizon_year}'
df[hhpop_horizon_col] = (df[households_horizon_col] * df['hhsz']).round(0).astype(int)

In [344]:
hhpop_horizon_sum_by_rgid_col = f'hhpop_{target_horizon_year}_sum_by_rgid'
df[hhpop_horizon_sum_by_rgid_col] = df.groupby('RGID')[hhpop_horizon_col].transform('sum')

In [347]:
df = df.merge(targets_rgid[['RGID', factored_hhpop_horizon_col]].rename(columns={factored_hhpop_horizon_col: f'rgid_factored_hhpop_{target_horizon_year}'}), on='RGID', how='left')

In [348]:
df['hhpop_factor'] = df[f'rgid_factored_hhpop_{target_horizon_year}'] / df[hhpop_horizon_sum_by_rgid_col]

In [350]:
hhpop_factored_horizon_col = f'hhpop_factored_{target_horizon_year}'
df[hhpop_factored_horizon_col] = (df[hhpop_horizon_col] * df['hhpop_factor']).round(0).astype(int)

In [351]:
df.head()

,target_id,start,units_chg,units_chg_adj,RGID,county_id,dec_total_pop,dec_units,dec_hh,dec_gq,...,dec_hhpop_by_rgid,dec_hh_by_rgid,dec_hhsz_by_rgid,dec_hhsz,hhsz,hhpop_2044,hhpop_2044_sum_by_rgid,rgid_factored_hhpop_2044,hhpop_factor,hhpop_factored_2044
0,1,2019,35000,34580,1,53033,151854,64688,60953,1440,...,857511,406580,2.109083,2.467705,2.340073,221376,1104682,1152938,1.043683,231046
1,2,2019,112000,109453,1,53033,737015,368308,345627,29918,...,857511,406580,2.109083,2.045838,1.940026,883306,1104682,1152938,1.043683,921892
2,3,2019,12000,11996,2,53033,77245,28050,27057,916,...,752795,286907,2.623829,2.821044,2.709411,103727,990961,982441,0.991402,102835
3,4,2019,5800,5457,2,53033,28956,12682,12006,289,...,752795,286907,2.623829,2.387723,2.293237,39767,990961,982441,0.991402,39425
4,5,2019,7500,7428,2,53033,52066,20785,19874,518,...,752795,286907,2.623829,2.593741,2.491102,67190,990961,982441,0.991402,66612


In [ ]:
reg_dec.head()

In [ ]:
ref = p.get_table('ref_projection')

In [ ]:
reg_gq_horizon = ref.loc[ref.variable == 'GQ Pop', str(target_horizon_year)].item()

In [ ]:
df['dec_gq_pct'] = df['dec_gq'] / reg_dec_gq

gq_horizon_col = f'gq_{target_horizon_year}'
df[gq_horizon_col] = (df['dec_gq_pct'] * reg_gq_horizon).round(0).astype(int)

In [ ]:
total_pop_horizon_col = f'totalpop_{target_horizon_year}'
df[total_pop_horizon_col] = df[hhpop_horizon_col] + df[gq_horizon_col]

In [ ]:
df.head()